## Auto- and crosscorrelation functions for spike trains
* https://nest-simulator.readthedocs.io/en/latest/auto_examples/cross_check_mip_corrdet.html



In [2]:
import nest
import numpy as np


In [3]:

def corr_spikes_sorted(spike1, spike2, tbin, tau_max, resolution):
    tau_max_i = int(tau_max / resolution)
    tbin_i = int(tbin / resolution)

    cross = np.zeros(int(2 * tau_max_i / tbin_i + 1), "d")

    j0 = 0

    for spki in spike1:
        j = j0
        while j < len(spike2) and spike2[j] - spki < -tau_max_i - tbin_i / 2.0:
            j += 1
        j0 = j

        while j < len(spike2) and spike2[j] - spki < tau_max_i + tbin_i / 2.0:
            cross[int((spike2[j] - spki + tau_max_i + 0.5 * tbin_i) / tbin_i)] += 1.0
            j += 1

    return cross


nest.ResetKernel()

In [4]:

resolution = 0.1  # Computation step size in ms
T = 100000.0  # Total duration
delta_tau = 10.0
tau_max = 100.0  # ms correlation window
t_bin = 10.0  # ms bin size
pc = 0.5
nu = 100.0

nest.local_num_threads = 30
nest.resolution = resolution
nest.overwrite_files = True
nest.rng_seed = 12345



Oct 08 13:39:55 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.


In [5]:

# Set up network, connect and simulate
mg = nest.Create("mip_generator")
mg.set(rate=nu, p_copy=pc)

cd = nest.Create("correlation_detector")
cd.set(tau_max=tau_max, delta_tau=delta_tau)

sr = nest.Create("spike_recorder", params={"time_in_steps": True})

pn1 = nest.Create("parrot_neuron")
pn2 = nest.Create("parrot_neuron")


nest.Connect(mg, pn1)
nest.Connect(mg, pn2)
nest.Connect(pn1, sr)
nest.Connect(pn2, sr)

nest.Connect(pn1, cd, syn_spec={"weight": 1.0, "receptor_type": 0})
nest.Connect(pn2, cd, syn_spec={"weight": 1.0, "receptor_type": 1})

nest.Simulate(T)


Oct 08 13:40:07 NodeManager::prepare_nodes [Info]: 
    Preparing 63 nodes for simulation.

Oct 08 13:40:07 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 63
    Simulation time (ms): 100000
    Number of OpenMP threads: 30
    Number of MPI processes: 1

Oct 08 13:40:08 SimulationManager::run [Info]: 
    Simulation finished.


In [6]:

n_events_1, n_events_2 = cd.n_events

lmbd1 = (n_events_1 / (T - tau_max)) * 1000.0
lmbd2 = (n_events_2 / (T - tau_max)) * 1000.0

spikes = sr.get("events", "senders")

sp1 = spikes[spikes == 4]
sp2 = spikes[spikes == 5]

# Find crosscorrelation
cross = corr_spikes_sorted(sp1, sp2, t_bin, tau_max, resolution)

print("Crosscorrelation:")
print(cross)
print("Sum of crosscorrelation:")
print(sum(cross))

Crosscorrelation:
[       0.        0.        0.        0.        0.        0.        0.
        0.        0.        0. 24028580.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.]
Sum of crosscorrelation:
24028580.0
